In [ ]:
import mlflow
import re
import numpy as np
from astrovision.data import SatelliteImage
import yaml
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

In [ ]:
%env MLFLOW_TRACKING_URI=https://projet-slums-detection-128833.user.lab.sspcloud.fr
%env MLFLOW_S3_ENDPOINT_URL=https://minio.lab.sspcloud.fr
model_name = "test"
model_version = "2"
model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

In [ ]:
artifact_uri = mlflow.get_run(model.metadata.run_id).info.artifact_uri

In [ ]:
dir(model)

In [ ]:
mlflow.artifacts.download_artifacts(
    artifact_uri=mlflow.get_run(model.metadata.run_id).info.artifact_uri,
)

In [ ]:
with open(
    f"{mlflow.get_run(model.metadata.run_id).info.artifact_uri}/model/code/metrics-normalization.yaml"
) as f:
    params = yaml.safe_load(f)

In [ ]:
mlflow.artifacts.load_dict()

In [ ]:
matches = re.findall(
    r'"artifact_path": "([^"]+)"',
    mlflow.get_run(model.metadata.run_id).data.tags["mlflow.log-model.history"],
)

In [ ]:
mlflow.get_run(model.metadata.run_id).data.tags["mlflow.log-model.history"]

In [ ]:
matches

In [ ]:
dir(mlflow.get_run(model.metadata.run_id).info)

In [ ]:
mlflow.get_run(model.metadata.run_id).info.artifact_uri

In [ ]:
mlflow.artifacts.load_dict(artifact_uri + "/config.json")

In [ ]:
si = SatelliteImage.from_raster(
    file_path="/vsis3/projet-slums-detection/data-preprocessed/patchs/BDTOPO/segmentation/PLEIADES/MAYOTTE/2022/250/train/ORT_976_2022_0532_8588_U38S_8Bits_0002.jp2",
    dep=None,
    date=None,
    n_bands=3,
)

In [ ]:
normalization_mean = 0
normalization_std = 0

In [ ]:
test_transform = A.Compose(
    [
        # TODO: Calculer moyenne et variance sur toutes les images
        A.Normalize(
            max_pixel_value=255.0,
            mean=normalization_mean,
            std=normalization_std,
        ),
        ToTensorV2(),
    ]
)

input = test_transform(image=np.transpose(si.array, [1, 2, 0]))

In [ ]:
model.predict(np.random.uniform(size=[1, 28, 28]).astype(np.float32))